# Enable APIs (Container, Kubernetes) in Google Cloud Platform

# Create cluster in console

# Install Google Cloud SDK

# Configure Docker to use gcloud tool to authenticate requests
> gcloud auth configure-docker

# Download auth token key from console and put it in the directory
> API & Services -> Credentials: create credentials > Service account key

# Fetch cluster endpoint and auth data and update kubeconfig entry

> gcloud container clusters get-credentials mm-docker-models-gke --zone us-east4-a [--project <project name>]

# Configure firewall to allow the node port range (30000-32767)
> gcloud compute firewall-rules create gke-rule --allow tcp:30000-32767


# Configure 'GCP' section in cli.properties


In [1]:
from mm_docker_lib import *
initConfig("GCP")

Loading configuration properties...
key file: sas-tyfrec-209092a0e3d2.json
project name: sas-tyfrec
Login into GCP GCR...
Login GCP GCR succeeded!
Initializing kubernetes configuration... gke_sas-tyfrec_us-central1-a_eyap-mas
Initialized kubernetes configuration.
  verbose: True
  model.repo.host: http://eyap-mas.gtp-americas.sashq-d.openstack.sas.com
  provider.type: GCP
  base.repo: gcr.io/sas-tyfrec/
  base.repo.web.url: console.cloud.google.com/gcr/images/sas-tyfrec
  kubernetes.context: gke_sas-tyfrec_us-central1-a_eyap-mas


True

In [2]:
listmodel("svm")

Getting model list...
Model name svm (pipeline 1)
Model UUID 56c91200-8b36-4139-afcd-8958683546df
Model version 1.2
Project name HMEQ
Score Code Type ds2MultiType
Image URL (not verified): gcr.io/sas-tyfrec/svm-pipeline-1_56c91200-8b36-4139-afcd-8958683546df:latest
Guides: > python mm_docker_cli.py publish <uuid>
Guides: > python mm_docker_cli.py launch <image url>
Guides: > python mm_docker_cli.py score <image url> <input file>


In [3]:
# returns the docker image URL of the container we created and pushed to our GCP repo
container = publish("56c91200-8b36-4139-afcd-8958683546df")

Images folder: /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images
Model zip file has been downloaded at /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images/model-56c91200-8b36-4139-afcd-8958683546df/model-56c91200-8b36-4139-afcd-8958683546df.zip
SVM (Pipeline 1) 1.2 ds2MultiType
The name has been normalized to svm-pipeline-1
Template folder: /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/template
Copying astore from shared directory...
Make sure that the shared directory has been mounted locally
For Windows, the local directory will be at z drive
For Linux, the local directory will be /astore
/astore/_99J390JZGMNE1NI0CQHB384QQ.astore
/net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images/model-56c91200-8b36-4139-afcd-8958683546df
Building image...
svm-pipeline-1_56c91200-8b36-4139-afcd-8958683546df:1.2
svm-pipeline-1_56c91200-8b36-4139-afcd-8958683546df:latest
Tag the image into a repository sha256:0705122764
Docker repository URL:  gcr.io/sas-tyfrec/
gcr.io/sas-tyfrec/

In [4]:
# launches the container into GCP Kubernetes engine.
# Returns a tuple of the form: (kubernetes-deployment-id, http://URL-of-service:1234)
deployment = launch(container)

Validating image repository url...
Completed validation.
Launching container instance...
gcr.io/sas-tyfrec/svm-pipeline-1_56c91200-8b36-4139-afcd-8958683546df:latest
svm-pipeline-1
Deployment created. 
Deployment name:  svm-pipeline-1-8m53un
Service created.
Getting service url...
Service URL:  http://35.188.175.53:31302
Checking whether the instance is up or not...
1 ==Sleep 10 seconds...
Checking whether the instance is up or not...
Instance is up!
Guides: > python mm_docker_cli.py execute  http://35.188.175.53:31302 <input file>
Guides: > python mm_docker_cli.py stop svm-pipeline-1-8m53un


In [5]:
# Scores the input file against the scoring service.
# Returns an id of the score
test_id = execute(deployment[1], "hmeq_test.csv")

Performing scoring in the container instance...
service_url: http://35.188.175.53:31302
csv_file: hmeq_test.csv
{'id': '1553275146.9801204', 'status': 201}
The test_id from score execution: 1553275146.9801204
Guides: > python mm_docker_cli.py query http://35.188.175.53:31302/ 1553275146.9801204


In [6]:
query(service_url=deployment[1],test_id=test_id)

service_url: http://35.188.175.53:31302
test_id: 1553275146.9801204
The test result has been retrieved and written into file 1553275146.9801204.csv
Head is the first 5 lines
EM_CLASSIFICATION,EM_EVENTPROBABILITY,EM_PROBABILITY,I_BAD,P_BAD0,P_BAD1,_P_,_WARN_
0                               ,5.807186e-10,1.0,0                               ,1.0,5.807186e-10,1.0,
0                               ,6.6705796e-10,1.0,0                               ,1.0,6.6705796e-10,1.0,
0                               ,5.7110605e-10,1.0,0                               ,1.0,5.7110605e-10,1.0,
1                               ,0.82900435,0.82900435,1                               ,0.17099567,0.82900435,-1.5353535,
Guides: 1) remember to stop instance after uage. You can find the deployment name by running
    > kubectl get deployment
Then execute: > python mm_docker_cli.py stop <deployment_name>
Guides: 2) if result file includes error message, you could find the pod name and debug inside the instance as below

'1553275146.9801204.csv'

In [7]:
stop(deployment_name=deployment[0])

svm-pipeline-1-8m53un
Deleting service svm-pipeline-1-8m53un
deleted svc/svm-pipeline-1-8m53un from ns/default
Deleting app deployment... svm-pipeline-1-8m53un
Deletion succeeded


In [8]:
score(container, "hmeq_test.csv")

Validating image repository url...
Completed validation.
Launching container instance...
gcr.io/sas-tyfrec/svm-pipeline-1_56c91200-8b36-4139-afcd-8958683546df:latest
svm-pipeline-1
Deployment created. 
Deployment name:  svm-pipeline-1-5frk68
Service created.
Getting service url...
Service URL:  http://35.188.175.53:31267
Checking whether the instance is up or not...
1 ==Sleep 10 seconds...
Checking whether the instance is up or not...
Instance is up!
Guides: > python mm_docker_cli.py execute  http://35.188.175.53:31267 <input file>
Guides: > python mm_docker_cli.py stop svm-pipeline-1-5frk68
Performing scoring in the container instance...
service_url: http://35.188.175.53:31267
csv_file: hmeq_test.csv
{'id': '1553275173.045694', 'status': 201}
The test_id from score execution: 1553275173.045694
Guides: > python mm_docker_cli.py query http://35.188.175.53:31267/ 1553275173.045694
service_url: http://35.188.175.53:31267
test_id: 1553275173.045694
The test result has been retrieved and wr